In [2]:
import time
import datetime
import serial
import serial.tools.list_ports

import os
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import shelve

def get_datetime_string():
    return time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())

def get_filename_with_tkinter():
    shelfFile = shelve.open('mydata')

    # try to open saved variables from shelfFile, if the saved variables are not there,
    # set the last open variable to "/"
    try:
        last_open_dir = shelfFile['last_open_dir']
    except KeyError:
        last_open_dir = "/"

    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)
    this_file =  Path(filedialog.askopenfilename(initialdir = last_open_dir,title = "Select file",filetypes = (("CSV files","*.csv"),("Text files","*.txt"),("all files","*.*"))))
    root.destroy()

    # save the last opened folder to the shelfFile
    shelfFile['last_open_dir'] = this_file.parent
    shelfFile.close()
    
    return this_file

def create_default_filename(comport, baud):
    return f"{get_datetime_string()}_serial_log_{comport}_{baud}.csv"

# create_default_filename("COM4", 9600)

def get_file_save_name_with_tkinter(default_filename=""):
    shelfFile = shelve.open('mydata')

    # try to open saved variables from shelfFile, if the saved variables are not there,
    # set the last open variable to "/"
    try:
        last_open_dir = shelfFile['last_open_dir']
    except KeyError:
        last_open_dir = "/"

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    this_file =  Path(filedialog.asksaveasfilename(initialdir = last_open_dir, initialfile = default_filename, title = "Save log file",filetypes = (("CSV files","*.csv"),("Text files","*.txt"),("all files","*.*"))))
    root.destroy()

    # save the last opened folder to the shelfFile
    shelfFile['last_open_dir'] = this_file.parent
    shelfFile.close()
    
    return this_file

# get_file_save_name_with_tkinter(create_default_filename("COM4", 9600))

def get_available_com_ports():
    print("Finding ports...")
    ports = serial.tools.list_ports.comports()
#     print("Available ports:")
#     for i in ports:
#         print(" - " + i.name)
    return ports

#todo - add default value
def get_response_from_menu_list(items):
    num_items = len(items)
    print(f"Pick an item between 1 and {num_items}")
    b_correct_input = False
    while (not b_correct_input):
        response = input("Select >>")
        if not response.isdecimal():
            print("Please input a number")
            continue
        if int(response) > num_items:
            print("Number too high")
            continue
        if int(response) < 1:
            print("Number too low")
            continue
        b_correct_input = True
    return int(response)

# get_response_from_menu_list(["a", "b", "c", "d"])

def print_menu(items):
    for i in range(len(items)):
        print(f"{i+1}: {items[i]}")
        
def get_item_from_menu(items, title="Select an item:", default=""):
    # default is the line of the default item
    print("")
    print(title)
    print_menu(items)

    num_items = len(items)
    print(f"Pick an item between 1 and {num_items}")
    
    if default != "":
        print(f"Press enter for default value: {default}")
        
    b_correct_input = False
    while (not b_correct_input):
        response = input("Select >>")
#         print(bytes(response))
        if (response == "") & (default != ""):
            response = default
            print("Default selected")
            break
        if not response.isdecimal():
            print("Please input a number")
            continue
        if int(response) > num_items:
            print("Number too high")
            continue
        if int(response) < 1:
            print("Number too low")
            continue
        b_correct_input = True
    
    selected_line = int(response)
    item_out = items[selected_line - 1]
    print(f"{item_out} selected")
    return item_out     
        
# print_menu(["a", "b", "c", "d"])

# ser.close()
# ser.close()
# ser.port(cbo_ports.get())
# ser.baudrate(cbo_speed.get())
# ser.open()

#TODO - this does not replace every slash in a text string such as \dev\tty4, it onle produces -dev\tty4
def create_default_filename(comport, baud):
    comport = comport.replace("\\","-",30) # replaces up to 30 slashes in filename
    return f"{get_datetime_string()}_{comport}_{baud}.txt"
    

In [7]:
try:
    ser.close() # close the serial port if it's open
except NameError:
    pass
    
serial_speeds = (300,1200,2400,4800,9600,19200,38400,57600,74880,115200,230400)

available_ports = get_available_com_ports() # finds the com ports on the system

if len(available_ports) == 0:
    print("There are no com ports available, program closing")
    quit()

serial_speeds = (300,1200,2400,4800,9600,19200,38400,57600,74880,115200,230400)

port_to_open = get_item_from_menu(available_ports, "Select a Serial Port").name # creates a menu to select the serial port from

serial_speed = get_item_from_menu(serial_speeds, "Select a baud rate") # creates a menu to select the serial speed from

log_filename = get_file_save_name_with_tkinter(create_default_filename(port_to_open, serial_speed))
print(f"Saving logfile as {log_filename}...")

with open(log_filename,'w') as f_out:
    f_out.write('Time,data\n')
                                               
print(f"\nOpening Serial port {port_to_open} at {serial_speed} baud...\n")
ser = serial.Serial(port_to_open, serial_speed, timeout=4)
# ser.open()

for i in range(20):
    bytes_in = ser.read_until() #read the data from the serial port, strip the line feed from it, then convert from bytes to a string
    data_in = bytes_in.strip().decode() #read the data from the serial port, strip the line feed from it, then convert from bytes to a string
    time_in = datetime.datetime.now().isoformat()
    log_line = bytes_in
#     log_line = time_in + "," + str(data_in) + ',' + data_in
    print(log_line) #read until end of line
    with open(log_filename,'a') as f_out:
        f_out.write(log_line + '\n')
    
print("Closing port...")
ser.close()

Finding ports...

Select a Serial Port
1: COM13 - Prolific USB-to-Serial Comm Port (COM13)
2: COM14 - USB Serial Device (COM14)
Pick an item between 1 and 2


Select >> 2


COM14 - USB Serial Device (COM14) selected

Select a baud rate
1: 300
2: 1200
3: 2400
4: 4800
5: 9600
6: 19200
7: 38400
8: 57600
9: 74880
10: 115200
11: 230400
Pick an item between 1 and 11


Select >> 2


1200 selected
Saving logfile as C:\Users\ed.bisdee\OneDrive - Hyperdrive Innovation 365\Documents\_code\tenma-multimeter-interface\2021-10-11_15-20-52_COM14_1200.txt...

Opening Serial port COM14 at 1200 baud...

b'0.000000\r\n'


TypeError: can't concat str to bytes

In [ ]:

#todo - add default value
def get_response_from_menu_list(items):
    num_items = len(items)
    print(f"Pick an item between 1 and {num_items}")
    b_correct_input = False
    while (not b_correct_input):
        response = input("Select >>")
        if not response.isdecimal():
            print("Please input a number")
            continue
        if int(response) > num_items:
            print("Number too high")
            continue
        if int(response) < 1:
            print("Number too low")
            continue
        b_correct_input = True
    return int(response)

# get_response_from_menu_list(["a", "b", "c", "d"])

def print_menu(items):
    for i in range(len(items)):
        print(f"{i+1}: {items[i]}")
        
def get_item_from_menu(items, title="Select an item:", default=""):
    # default is the line of the default item
    print("")
    print(title)
    print_menu(items)

    num_items = len(items)
    print(f"Pick an item between 1 and {num_items}")
    
    if default != "":
        print(f"Press enter for default value: {default}")
        
    b_correct_input = False
    while (not b_correct_input):
        response = input("Select >>")
#         print(bytes(response))
        if (response == "") & (default != ""):
            response = default
            print("Default selected")
            break
        if not response.isdecimal():
            print("Please input a number")
            continue
        if int(response) > num_items:
            print("Number too high")
            continue
        if int(response) < 1:
            print("Number too low")
            continue
        b_correct_input = True
    
    selected_line = int(response)
    item_out = items[selected_line - 1]
    print(f"{item_out} selected")
    return item_out   

get_item_from_menu(["a", "b", "c", "d"], title="cheese", default=3)

In [22]:


create_default_filename("/dev/tty4", 9600)


'2021-06-14_13-07-32_/dev/tty4_9600.txt'

In [20]:
def save_variable(var, var_name, shelfFileName = 'mydata'):
    shelfFile = shelve.open(shelfFileName)
    # save the last opened folder to the shelfFile
    shelfFile[var_name] = var
    shelfFile.close()

def open_saved_variable(var_name, shelfFileName = 'mydata'):
    shelfFile = shelve.open(shelfFileName)
    # try to open saved variables from shelfFile, if the saved variables are not there,
    # set the last open variable to "/"
    try:
        var_temp = shelfFile[var_name]
        shelfFile.close()
        return var_temp
    except KeyError:
        return ''
    
foo = 'cheese'
name = 'foo'

save_variable(foo,name)
save_variable(foo,name,'data2')

print(open_saved_variable('foo'))
print(open_saved_variable('ham','data2'))


cheese
cheese


In [14]:



def get_item_from_menu(items, title="Select an item:", default=""):
    # default is the line of the default item
    print("")
    print(title)
    print_menu(items)

    num_items = len(items)
    print(f"Pick an item between 1 and {num_items}")
    
    b_correct_input = False
    if num_items == 1:
        print(f"Only '{items[0]}' available")
        response = 1
        b_correct_input = True
        
    if default != "":
        print(f"Press enter for default value: {default}")
        
    while (not b_correct_input):
        response = input("Select >>")
#         print(bytes(response))
        if (response == "") & (default != ""):
            response = default
            print("Default selected")
            break
        if not response.isdecimal():
            print("Please input a number")
            continue
        if int(response) > num_items:
            print("Number too high")
            continue
        if int(response) < 1:
            print("Number too low")
            continue
        b_correct_input = True
    
    selected_line = int(response)
    item_out = items[selected_line - 1]
    print(f"{item_out} selected")
    return item_out     
        
# print_menu(["a", "b", "c", "d"])

# get_item_from_menu(['a','b','c'])

get_item_from_menu(['a'], title="Select an item:", default="")


Select an item:
1: a
Pick an item between 1 and 1
Only 'a' available
a selected


'a'

In [5]:

# write header
with open('testlog.csv', 'w') as x:
    x.write('time,data\n')
    
# write line
for i in range(10):
    with open('testlog.csv', 'a') as x:
        x.write(f'{i},bees\n')

In [3]:
# write line
for i in range(10):
    with open('testlog.txt', 'a') as x:
        x.write(f'line: {i}\n')